In [1]:
from gen_ai_hub.orchestration.utils import load_text_file 

# Load the CV file content 
cv_file_path = "cv.txt" # Specify the correct path to the CV file 
cv_content = load_text_file(cv_file_path) 

In [2]:
from gen_ai_hub.orchestration.models.message import SystemMessage, UserMessage 
from gen_ai_hub.orchestration.models.template import Template, TemplateValue 
from gen_ai_hub.orchestration.models.response_format import ResponseFormatJsonSchema

json_schema = {
    "title": "Search Query",
    "type": "object",
    "properties": {
            "search_query": {
            "type": "string",
            "description": "Query that is optimized web search."
        },
            "justification": {
            "type": "string",
            "description": "Why this query is relevant to the user's request."
        }
    }
}

# Define the template for resume screening 
template = Template( 
    messages=[ 
        SystemMessage("""You are a helpful AI assistant with deep knowledge in sciences and web searching"""), 

        UserMessage( 
            "Here is a questions: {{?question}}" 
        ), 
    ], 
    defaults=[ 
        TemplateValue(name="question", value="Some intersting science question"), 
    ],
    response_format = ResponseFormatJsonSchema(name="person", description="person mapping", schema=json_schema),
) 

In [3]:
from gen_ai_hub.orchestration.models.llm import LLM
from gen_ai_hub.orchestration.models.config import OrchestrationConfig 

# List of models to use 
models = [ 
    LLM(name="gpt-4o", version="latest", parameters={"max_tokens": 1000, "temperature": 0.6}), 
    LLM(name="gemini-1.5-pro", version="latest", parameters={"max_tokens": 1000, "temperature": 0.6}),
] 

# Create configurations for each model 
configs = [] 
for model in models: 
    # Create orchestration config for each model 
    config = OrchestrationConfig( 
        template=template,
        llm=model,
    ) 
    configs.append(config) 

print("Model configurations created successfully:") 

Model configurations created successfully:


In [4]:
from gen_ai_hub.orchestration.service import OrchestrationService

ORCH_URL = "https://api.ai.intprod-eu12.eu-central-1.aws.ml.hana.ondemand.com/v2/inference/deployments/d809d5b40514e546"
question = "How does Calcium CT score relate to high cholesterol?"

# Initialize an empty list to store the responses 
responses = [] 

# Iterate through each config and get the response using the filtered input 
for i, config in enumerate(configs): 
    orchestration_service = OrchestrationService(api_url=ORCH_URL, config=config)
    # Run orchestration with the provided input (for example, candidate resume content) 
    result = orchestration_service.run(template_values=[ 
        TemplateValue(name="question", value=question)
    ])
    # Extract the response content 
    response = result.orchestration_result.choices[0].message.content
    # Append the response to the responses list 
    responses.append({ 
        "model": models[i].name,    # Store model name 
        "response": response        # Store the corresponding model response 
    }) 

# Store the responses in a text file 
with open("model_responses.txt", "w") as file: 
    for response_data in responses: 
        file.write(f"Response from model {response_data['model']}:\n") 
        file.write(f"{response_data['response']}\n") 
        file.write("-" * 80 + "\n") # Add a separator between model responses